<a href="https://colab.research.google.com/github/Maram-Elsayed/Panelty-Function-With-Gradient-Descent/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import numpy as np
import PIL
import cv2
import keras
from PIL import Image
import os, os.path
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy import ndimage, misc
from keras.utils import to_categorical
 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

In [0]:
imageDir_train = ["drive/Colab/daisy_0/","drive/Colab/dandelion_1/","drive/Colab/rose_2/","drive/Colab/sunflower_3/","drive/Colab/tulip_4/"]

y=0
train_x=[]
flowers=np.array([['Pixels','Target']])
train_y=np.array([])
valid_image_extensions = [".jpg", ".jpeg", ".png"]
valid_image_extensions = [item.lower() for item in valid_image_extensions]
for f in range(len(imageDir_train)):
 image_path_list_train=[]
 for file in os.listdir(imageDir_train[f]):    
    extension = os.path.splitext(file)[1]
    if extension.lower() not in valid_image_extensions:
        continue
    image_path_list_train.append(os.path.join(imageDir_train[f], file))



 for imagePath in image_path_list_train:
   i= np.array(Image.open(imagePath))
   res = cv2.resize(i, dsize=(320, 230), interpolation=cv2.INTER_CUBIC)
   iar=np.asarray(res)
   flowers=np.append(flowers,[[iar,imageDir_train[f][-2:-1]]] ,0)
   y+=1
   

In [0]:
flowers = np.delete(flowers, (0), axis=0)

In [0]:
np.random.shuffle(flowers)

In [0]:
for x in range(len(flowers)):
 train_y=np.append(train_y,[flowers[x][1]])
for x in range(len(flowers)):
 train_x.append(flowers[x][0])
train_x=np.array(train_x)


In [0]:
train_x = train_x.reshape(-1, 320,230, 3)
train_x = train_x.astype('float32')
train_x /= 255


In [0]:
train_y= to_categorical(train_y)

In [0]:
valid_x=train_x[-300:]
train_x=train_x[:-300]
valid_y=train_y[-300:]
train_y=train_y[:-300]


test_x=train_x[-300:]
train_x=train_x[:-300]
test_y=train_y[-300:]
train_y=train_y[:-300]

In [0]:
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

fashion_model = Sequential()
fashion_model.add(Convolution2D(64, kernel_size=(5, 5),activation='linear',input_shape=(320,230,3),padding='same'))
fashion_model.add(LeakyReLU(alpha=0.01))
fashion_model.add(MaxPooling2D((2, 2),padding='same'))
fashion_model.add(Convolution2D(64, (5, 5), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.01))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Convolution2D(128, (5, 5), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.01))                  
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Convolution2D(516, (5, 5), activation='linear',padding='same'))
fashion_model.add(LeakyReLU(alpha=0.01))                  
fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation='linear'))
fashion_model.add(LeakyReLU(alpha=0.001))  
fashion_model.add(Dense(128, activation='linear'))
fashion_model.add(LeakyReLU(alpha=0.001))  
fashion_model.add(Dense(128, activation='linear'))
fashion_model.add(LeakyReLU(alpha=0.001))  
fashion_model.add(Dense(5, activation='softmax'))

In [0]:
fashion_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [0]:


callbacks = [
   keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=3,
                              verbose=1, mode='auto')
]
fashion_train = fashion_model.fit(train_x, train_y, batch_size=64,epochs=30,verbose=1,validation_data=(valid_x, valid_y),callbacks=callbacks)

Train on 2410 samples, validate on 300 samples
Epoch 1/30
2410/2410 [==============================] - 34s 14ms/step - loss: 1.5888 - acc: 0.2398 - val_loss: 1.4725 - val_acc: 0.2867
Epoch 2/30
2410/2410 [==============================] - 33s 14ms/step - loss: 1.3879 - acc: 0.3934 - val_loss: 1.3321 - val_acc: 0.4100
Epoch 3/30
2410/2410 [==============================] - 33s 14ms/step - loss: 1.2265 - acc: 0.4739 - val_loss: 1.1901 - val_acc: 0.4933
Epoch 4/30
2410/2410 [==============================] - 33s 14ms/step - loss: 1.1466 - acc: 0.5253 - val_loss: 1.1047 - val_acc: 0.5367
Epoch 5/30
2410/2410 [==============================] - 33s 14ms/step - loss: 1.0986 - acc: 0.5606 - val_loss: 1.0348 - val_acc: 0.5800
Epoch 6/30
2410/2410 [==============================] - 33s 14ms/step - loss: 1.0447 - acc: 0.5913 - val_loss: 1.0032 - val_acc: 0.6167
Epoch 7/30
2410/2410 [==============================] - 33s 14ms/step - loss: 0.9913 - acc: 0.5988 - val_loss: 1.0587 - val_acc: 0.5433
E

In [0]:
test_eval = fashion_model.evaluate(test_x, test_y, verbose=0)

print('Test accuracy:', test_eval[1])
